In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
classes = ('T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot')
num_classes = len(classes)

In [3]:
train_transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(28,padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [4]:
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=train_transformations, download=True)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=test_transformations, download=True)

In [5]:
#trainset, valset = train_test_split(trainset, test_size=0.2, random_state=42)

In [6]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=0)
#valloader = torch.utils.data.DataLoader(valset, batch_size=4, shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=0)

In [7]:
class Net(nn.Module):
    def __init__(self, in_chan, out_chan):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_chan, out_channels=out_chan, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(num_features=out_chan)
        self.drop = nn.Dropout(p=0.2)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.dropout(self.drop(x), training=self.training)
        x = self.relu(x)
        return x

class SimpleNet(nn.Module):
    def __init__(self, num_class=10):
        super(SimpleNet, self).__init__()
        self.net1 = Net(1,16)
        self.net2 = Net(16,32)
        self.net3 = Net(32,64)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.net4 = Net(64,64)
        self.net5 = Net(64,64)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.net6 = Net(64,128)
        self.net7 = Net(128,128)
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        self.avgpool = nn.AvgPool2d(kernel_size=3)
        
        self.net = nn.Sequential(self.net1,self.net2,self.net3,self.pool1,self.net4,self.net5,self.pool2,self.net6,self.net7,self.pool3,self.avgpool)
        self.fc = nn.Linear(in_features=128,out_features=num_class)

    def forward(self, input):
        output = self.net(input)
        output = output.view(-1,128)
        output = self.fc(output)
        return output

In [8]:
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [9]:
def adjust_learning_rate(epoch):
    lr = 0.001

    if epoch > 50:
        lr = lr / 1000000
    elif epoch > 40:
        lr = lr / 100000
    elif epoch > 30:
        lr = lr / 10000
    elif epoch > 20:
        lr = lr / 1000
    elif epoch > 10:
        lr = lr / 100
    elif epoch > 5:
        lr = lr / 10

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

In [10]:
def train(num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            images, labels = data
            # Clear all accumulated gradients
            optimizer.zero_grad()
            # Predict classes using images from the test set
            outputs = model(images)
            # Compute the loss based on the predictions and actual labels
            loss = criterion(outputs, labels)
            # Backpropagate the loss
            loss.backward()
            # Adjust parameters according to the computed gradients
            optimizer.step()
            # Update running loss
            running_loss += loss.item()
            # Print the metrics
            if i % 5000 == 4999:
                print("Epoch {}, Count: {}, Running Loss: {}".format(epoch+1, i+1,running_loss/5000))
        #adjust_learning_rate(epoch)

In [11]:
train(5)

Epoch 1, Count: 5000, Running Loss: 6235.261148370802
Epoch 1, Count: 10000, Running Loss: 10361.800297427922
Epoch 1, Count: 15000, Running Loss: 14024.777436641045
Epoch 2, Count: 5000, Running Loss: 3456.4860220146365
Epoch 2, Count: 10000, Running Loss: 6771.258725560736
Epoch 2, Count: 15000, Running Loss: 9945.807636388461
Epoch 3, Count: 5000, Running Loss: 3125.2288926432375
Epoch 3, Count: 10000, Running Loss: 6124.01325264154
Epoch 3, Count: 15000, Running Loss: 9092.903437818866
Epoch 4, Count: 5000, Running Loss: 2838.0045968536288
Epoch 4, Count: 10000, Running Loss: 5636.685754989041
Epoch 4, Count: 15000, Running Loss: 8477.3511550806
Epoch 5, Count: 5000, Running Loss: 2741.8221569583984
Epoch 5, Count: 10000, Running Loss: 5451.800809071981
Epoch 5, Count: 15000, Running Loss: 8091.801037339144


In [12]:
PATH = './fashion-MNIST_e5.pth'
torch.save(model.state_dict(), PATH)

In [13]:
"""net = SimpleNet()
net.load_state_dict(torch.load(PATH))"""

'net = SimpleNet()\nnet.load_state_dict(torch.load(PATH))'

In [40]:
def test(data, model_name=model):
    model_name.eval()
    test_acc = 0.0
    for i, (images, labels) in enumerate(data):
        # Predict classes using images from the test set
        outputs = model_name(images)
        _, prediction = torch.max(outputs.data, 1)
        
        test_acc += torch.sum(prediction == labels.data)

    # Compute the average acc and loss over all 10000 test images
    test_acc = round((test_acc / 10000).tolist(), 4)*100

    return test_acc

In [15]:
test(testloader)

44.66

In [16]:
"""for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)"""

'for name, param in model.named_parameters():\n    if param.requires_grad:\n        print(name, param.data)'

In [17]:
class_correct = list(0. for i in range(num_classes))
class_total = list(0. for i in range(num_classes))
with torch.no_grad():
    #i=0
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        """if i < 3:
            print(outputs)
            print(predicted)
            i+=1
        else:
            break"""
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(num_classes):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of T-shirt/top :  0 %
Accuracy of Trouser : 90 %
Accuracy of Pullover :  2 %
Accuracy of Dress : 45 %
Accuracy of  Coat : 74 %
Accuracy of Sandal :  0 %
Accuracy of Shirt : 85 %
Accuracy of Sneaker : 98 %
Accuracy of   Bag : 29 %
Accuracy of Ankle boot : 19 %


In [35]:
class NeuNet(nn.Module):
    def __init__(self):
        super(NeuNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.drop = nn.Dropout2d(p=0.2)
        self.fc1 = nn.Linear(in_features= 7 * 7 * 24, out_features=20)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=20, out_features=10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        #x = self.drop(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu2(x)
        #x = self.drop(x)   
        x = self.pool(x)
        x = x.view(-1, 7 * 7 * 24)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

In [36]:
net = NeuNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [37]:
for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        img, label = data
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(img)
        
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5000 == 4999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5000))
            running_loss = 0.0

print('Finished Training')

[1,  5000] loss: 2.897
[1, 10000] loss: 1.883
[1, 15000] loss: 1.666
[2,  5000] loss: 1.556
[2, 10000] loss: 1.461
[2, 15000] loss: 1.402
[3,  5000] loss: 1.325
[3, 10000] loss: 1.265
[3, 15000] loss: 1.237
[4,  5000] loss: 1.184
[4, 10000] loss: 1.160
[4, 15000] loss: 1.118
[5,  5000] loss: 1.123
[5, 10000] loss: 1.074
[5, 15000] loss: 1.082
Finished Training


In [38]:
class_correct = list(0. for i in range(num_classes))
class_total = list(0. for i in range(num_classes))
with torch.no_grad():
    #i=0
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        """if i < 3:
            print(outputs)
            print(predicted)
            i+=1
        else:
            break"""
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(num_classes):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of T-shirt/top : 74 %
Accuracy of Trouser : 97 %
Accuracy of Pullover : 77 %
Accuracy of Dress : 84 %
Accuracy of  Coat : 87 %
Accuracy of Sandal : 94 %
Accuracy of Shirt : 52 %
Accuracy of Sneaker : 94 %
Accuracy of   Bag : 98 %
Accuracy of Ankle boot : 95 %


In [30]:
PATH = './fashion-MNIST_NeuNetfirst.pth'
torch.save(net.state_dict(), PATH)

In [42]:
test(testloader,net)

85.65